# Hyper Parameter Tunning

머신러닝 알고리즘을 통해 학습을 진행할 때, 알고리즘의 다양한 파라미터를 통해 데이터에 대한 최적의 모델을 찾는 과정이 필요합니다. 메뉴얼한 방식으로 대략적인 알고리즘의 선택을 할 수는 있지만, 최적의 모델을 선택하기 위해서는 다양한 파라미터 조합을 통한 많은 학습이 필요하게 됩니다. 데이터 수, 피처 수, 파라미터 조합에 따라 최적화 수행 시간은 기하 급수적으로 늘어나게 됩니다.

## Grid Search & Randomized Search

파라미터에 의해서 생성할 수 있는 모든 파라미터의 조합을 생성하여 최적화 파라미터를 찾습니다. 전수를 테스트하는 Grid Search 방식은 Search Space 가 크기 때문에 조금만 파라미터가 커치면 학습해야하는 조합이 늘게 됩니다. 그래서 이보다는 모든 조합을 시도하는 대신 모든 반복에서 각 파라미터에 대해 랜덤 값을 선택하여 학습 조합 수를 줄이는 Randomized Search 방식이 자주 사용됩니다.

In [1]:
# 경고 메시지 출력 끄기
import warnings 
warnings.filterwarnings(action='ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import IPython

import sys

rseed = 22
import random
random.seed(rseed)

import numpy as np
np.random.seed(rseed)
np.set_printoptions(precision=3)
np.set_printoptions(formatter={'float_kind': "{:.3f}".format})

import pandas as pd
pd.set_option('display.max_rows', None) 
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)
pd.options.display.float_format = '{:,.5f}'.format

import sklearn

print(f"python ver={sys.version}")
print(f"pandas ver={pd.__version__}")
print(f"numpy ver={np.__version__}")
print(f"sklearn ver={sklearn.__version__}")

python ver=3.8.9 (default, Jun 12 2021, 23:47:44) 
[Clang 12.0.5 (clang-1205.0.22.9)]
pandas ver=1.2.4
numpy ver=1.19.5
sklearn ver=0.24.2


In [6]:
from sklearn import datasets, model_selection, svm, metrics
import os, joblib

# 데이터
n_samples = 1000
xs, ys = datasets.make_classification(
    n_samples=n_samples, # 데이터 수
    n_features=10, # X feature 수
    n_informative=3,
    n_classes=3, # Y class 수
    random_state=rseed) # 난수 발생용 Seed 값
print(f"data shape: xs={xs.shape}, ys={ys.shape}")
train_xs, test_xs, train_ys, test_ys = model_selection.train_test_split(
    xs, ys, test_size=0.3, shuffle=True, random_state=rseed)

# Set the parameters by cross-validation
# key: 파라미터 이름
# value: 최적화를 위한 파라미터 값 리스트
tuned_parameters = [
    {'kernel': ['rbf'], 'gamma': [1e-3, 1e-4], 'C': [1, 10, 100, 1000]},
    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}
]

scores = ['precision', 'recall']

for score in scores:
    print(f"Tuning hyper-parameters for {score}")
    
    # GridSearch Cross Validation 방식
    #clf = model_selection.GridSearchCV(svm.SVC(), tuned_parameters, cv=10, scoring=f"{score}_macro", n_jobs=-1)
    
    # Randomized Cross Validation 방식
    clf = model_selection.RandomizedSearchCV(svm.SVC(), tuned_parameters, cv=10, scoring=f"{score}_macro", n_jobs=-1)
    
    clf.fit(train_xs, train_ys)

    print(f"Best parameters set found on development set:\n {clf.best_params_}")
    print(f"\nBest estimator set found on development set:\n {clf.best_estimator_}")
    print(f"\nGrid scores on development set:")

    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print(f"{mean:0.3f} (+/-{std * 2:0.03f}) for {params}")

    print("\nDetailed classification report:")
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")

    y_true, y_pred = test_ys, clf.predict(test_xs)
    print(metrics.classification_report(y_true, y_pred))

data shape: xs=(1000, 10), ys=(1000,)
Tuning hyper-parameters for precision
Best parameters set found on development set:
 {'kernel': 'rbf', 'gamma': 0.001, 'C': 100}

Best estimator set found on development set:
 SVC(C=100, gamma=0.001)

Grid scores on development set:
0.736 (+/-0.101) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 1000}
0.706 (+/-0.101) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 10}
0.726 (+/-0.108) for {'kernel': 'linear', 'C': 1}
0.768 (+/-0.076) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 100}
0.115 (+/-0.003) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 1}
0.712 (+/-0.105) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 1}
0.724 (+/-0.111) for {'kernel': 'linear', 'C': 100}
0.726 (+/-0.106) for {'kernel': 'linear', 'C': 1000}
0.750 (+/-0.080) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}
0.724 (+/-0.111) for {'kernel': 'linear', 'C': 10}

Detailed classification report:
The model is trained on the full development set.
The scores are computed on the full evaluation set.
